In [152]:
INPUT_FILE = "training-1.in"
OUTPUT_FILE = "test.out"
SINGLE = 0
DOUBLE = 1
LEFT = "A"
RIGHT = "B"
ALL = "ALL"
DEBUG = 0

HARD_COST = 0
ENERGY_COST = 0
class Server():
    def __init__(self,server_id,server_type,cpu,mem,hard_cost,energy_cost):
        # 初始化配置
        self.server_id = server_id
        self.server_type = server_type
        self.cpu = cpu
        self.mem = mem
        self.hard_cost = hard_cost
        self.energy_cost = energy_cost
        # 剩余资源
        self.total_left_cpu = self.total_right_cpu = cpu/2
        self.total_left_mem = self.total_right_mem = mem/2
        # 已经使用的资源
        self.used_left_cpu = self.used_right_cpu = 0
        self.used_left_mem = self.used_right_mem = 0
        
        
    def cal_energy_cost(self):
        # 资源未使用则不计算电费
        if (self.used_left_cpu == 0) and (self.used_right_cpu == 0) and \
            (self.used_left_mem == 0) and (self.used_right_mem == 0):
                return 0
        else:
            return self.energy_cost
#         if (self.used_left_cpu + self.used_right_cpu == 0) and (self.used_left_mem + self.used_right_mem == 0):
#             return 0
#         else:
#             return self.energy_cost
        
    def __str__(self):
        # TODO
        return f"server id = {self.server_id}, server type = {self.server_type}\nused/total:\n"+ \
                f"left: CPU = {self.used_left_cpu}/{self.total_left_cpu}, MEM = {self.used_left_mem}/{self.total_left_mem}\n" + \
                f"right: CPU = {self.used_right_cpu}/{self.total_right_cpu}, MEM = {self.used_right_mem}/{self.total_right_mem}"
        
# 维护虚拟机id和对应的部署服务器上的部署位置：服务器id，节点位置
class VM():
    def __init__(self,server_id,way,vm_type):
        self.server_id = server_id
        self.way = way
        self.vm_type = vm_type
    
        
        
data = open(INPUT_FILE)
out = open(OUTPUT_FILE)
server_cnt = int(next(data))
# print(f"server_cnt = {server_cnt}")
# 服务器类型字典
server_dic = {}
for i in range(0,server_cnt):
    server_info = next(data).strip()
    server_info = server_info[1:-1]
    info_detail = server_info.split(",")
    one_server = {}
    server_type = info_detail[0]
    one_server["cpu"] = int(info_detail[1].strip())
    one_server["mem"] = int(info_detail[2].strip())
    one_server["hard_cost"] = int(info_detail[3].strip())
    one_server["energy_cost"] = int(info_detail[4].strip())
    server_dic[server_type] = one_server



# 根据虚拟机id找对应虚拟机的部署情况
get_vm_dic = {}
    

# 虚拟机类型字典
vm_dic = {}
vm_cnt = int(next(data))
# print(f"vm_cnt = {vm_cnt}")
for i in range(0,vm_cnt):
    vm_info = next(data).strip()
    vm_info = vm_info[1:-1]
    info_detail = vm_info.split(",")
    one_vm = {}
    vm_type = info_detail[0]
    one_vm["cpu"] = int(info_detail[1].strip())
    one_vm["mem"] = int(info_detail[2].strip())
    one_vm["deploy_type"] = int(info_detail[3].strip())
    vm_dic[vm_type] = one_vm

# 数据中心购买的服务器列表，里面的每个元素也是列表，表示这台服务器里部署的虚拟机ID
# 服务器id为在列表中的下标索引
server_center = []

days = int(next(data))
for day in range(days):
#     print(f"cur day = {day}")
    
    # 根据输出文件，购买服务器
    info = next(out).strip()
    purchase_info = info[1:-1].split(",")[0]
    purchase_type_cnt = int(info[1:-1].split(",")[1].strip())
#     print(f"purchase_type_cnt = {purchase_type_cnt}")
    for type_cnt in range(purchase_type_cnt):
        line = next(out).strip()
        line = line[1:-1]
        server_type = line.split(",")[0]
        server_cnt = int(line.split(",")[-1].strip())
        for i in range(server_cnt):
            server_info = server_dic[server_type]
            add_server = Server(len(server_center),server_type,server_info["cpu"],server_info["mem"],server_info["hard_cost"],server_info["energy_cost"])
            server_center.append(add_server)
            # 计算当天购买费用
            HARD_COST += add_server.hard_cost

    
    # 迁移操作
    line = next(out).strip()[1:-1]
    migration_cnt = int(line.split(",")[1].strip())
#     print(f"migration_cnt = {migration_cnt}")
    for i in range(migration_cnt):
        line = next(out).strip()[1:-1]
        line = line.split(",")
        vm_id = int(line[0])
        # 目标服务器id
        target_server_id = int(line[1])
        cur_vm = get_vm_dic[vm_id]
        # 原始服务器
        origin_server = server_center[cur_vm.server_id]
        # 目标服务器
        target_server = server_center[target_server_id]
        vm_type = cur_vm.vm_type
        vm_cpu = vm_dic[vm_type]["cpu"]
        vm_mem = vm_dic[vm_type]["mem"]
        if vm_dic[vm_type]["deploy_type"] == SINGLE:
            # 先释放源服务器资源
            if cur_vm.way == LEFT:
                origin_server.used_left_cpu -= vm_cpu
                origin_server.used_left_mem -= vm_mem
            elif cur_vm.way == RIGHT:
                origin_server.used_right_cpu -= vm_cpu
                origin_server.used_right_mem -= vm_mem
            # 占用新服务器资源
            # 目标服务器的部署节点
            target_deploy_way = line[2].strip()
            if target_deploy_way == LEFT:
                target_server.used_left_cpu += vm_cpu
                target_server.used_left_mem += vm_mem
            elif target_deploy_way == RIGHT:
                target_server.used_right_cpu += vm_cpu
                target_server.used_right_mem += vm_mem
            # 迁移后，更改对应虚拟机所属服务器
            get_vm_dic[vm_id] = VM(target_server_id,target_deploy_way,vm_type)    
            
        elif vm_dic[vm_type]["deploy_type"] == DOUBLE:
            # 先释放源服务器资源
            origin_server.used_left_cpu -= vm_cpu/2
            origin_server.used_left_mem -= vm_mem/2              
            origin_server.used_right_cpu -= vm_cpu/2
            origin_server.used_right_mem -= vm_mem/2                   
            # 占用新服务器资源
            target_server.used_left_cpu += vm_cpu/2
            target_server.used_left_mem += vm_mem/2              
            target_server.used_right_cpu += vm_cpu/2
            target_server.used_right_mem += vm_mem/2        
            # 迁移后，更改对应虚拟机所属服务器
            get_vm_dic[vm_id] = VM(target_server_id,ALL,vm_type)

    
    # 根据每一次虚拟机请求，模拟创建和删除
    req_cnt = int(next(data).strip())
    for i in range(req_cnt):
        line = next(data).strip()[1:-1]
        line = line.split(",")
        mode = line[0]
        # 处理add和del操作
        if mode == "add":
            out_line = next(out).strip()
            vm_type = line[1].strip()
            vm_id = int(line[2].strip())
            vm_cpu = vm_dic[vm_type]["cpu"]
            vm_mem= vm_dic[vm_type]["mem"]
            if DEBUG:
                print("=================")
                print(f"need_cpu = {vm_cpu}, need_mem = {vm_mem}, deploy_type = {vm_dic[vm_type]['deploy_type']}")
                print("=================")
            
            out_line = out_line[1:-1]
            out_line = out_line.split(",")
            server_id = int(out_line[0])
            # 找到对应部署的服务器
            cur_server = server_center[server_id]
            if vm_dic[vm_type]["deploy_type"] == SINGLE:
                way = out_line[1].strip()
                if way == LEFT:
                    # 判定能否部署
                    if (cur_server.used_left_cpu + vm_cpu <= cur_server.total_left_cpu) \
                        and (cur_server.used_left_mem + vm_mem <= cur_server.total_left_mem):
                        cur_server.used_left_cpu += vm_cpu
                        cur_server.used_left_mem += vm_mem
                    else:
                        raise Exception("部署后服务器负载超限！")
                elif way == RIGHT:
                     # 判定能否部署
                    if (cur_server.used_right_cpu + vm_cpu <= cur_server.total_right_cpu) \
                        and (cur_server.used_right_mem + vm_mem <= cur_server.total_right_mem):
                        cur_server.used_right_cpu += vm_cpu
                        cur_server.used_right_mem += vm_mem
                    else:
                        raise Exception("部署后服务器负载超限！")    
                # 标记该虚拟机的部署情况，方便之后释放
                get_vm_dic[vm_id] = VM(server_id,way,vm_type)
            elif vm_dic[vm_type]["deploy_type"] == DOUBLE:
                # 左右节点各索取一半资源
                # 左
                if (cur_server.used_left_cpu + vm_cpu/2 <= cur_server.total_left_cpu) \
                    and (cur_server.used_left_mem + vm_mem/2 <= cur_server.total_left_mem):
                    cur_server.used_left_cpu += vm_cpu/2
                    cur_server.used_left_mem += vm_mem/2
                else:
                    raise Exception("部署后服务器负载超限！")                
                # 右
                if (cur_server.used_right_cpu + vm_cpu/2 <= cur_server.total_right_cpu) \
                    and (cur_server.used_right_mem + vm_mem/2 <= cur_server.total_right_mem):
                    cur_server.used_right_cpu += vm_cpu/2
                    cur_server.used_right_mem += vm_mem/2
                else:
                    raise Exception("部署后服务器负载超限！")                  
                # 标记该虚拟机的部署情况，方便之后释放
                get_vm_dic[vm_id] = VM(server_id,ALL,vm_type)
            
#             print(f"{vm_id} 部署成功！")
#             print(cur_server)
            
        elif mode == "del":
            vm_id = int(line[1].strip())
            cur_vm = get_vm_dic[vm_id]
            vm_type = cur_vm.vm_type
            vm_cpu = vm_dic[vm_type]["cpu"]
            vm_mem = vm_dic[vm_type]["mem"]
            # 根据虚拟机类型释放对应资源
            server_id = cur_vm.server_id
            deploy_way = cur_vm.way
            cur_server = server_center[server_id]
            if deploy_way == LEFT:
                cur_server.used_left_cpu -= vm_cpu
                cur_server.used_left_mem -= vm_mem
            elif deploy_way == RIGHT:
                cur_server.used_right_cpu -= vm_cpu
                cur_server.used_right_mem -= vm_mem
            elif deploy_way == ALL:
                cur_server.used_left_cpu -= vm_cpu/2
                cur_server.used_left_mem -= vm_mem/2              
                cur_server.used_right_cpu -= vm_cpu/2
                cur_server.used_right_mem -= vm_mem/2     
            del get_vm_dic[vm_id]
#             print(f"{vm_id} 对应虚拟机删除成功！")
     
    # 计算每天有负载的服务器耗电
    for server in server_center:
        ENERGY_COST += server.cal_energy_cost()
        

print("********************")
print(f"HARD_COST = {HARD_COST}, ENERGY_COST = {ENERGY_COST}, TOTAL_COST = {HARD_COST+ENERGY_COST}")
    


Exception: 部署后服务器负载超限！

In [94]:
len(server_center)

41

In [96]:
print(server_center[3])

server type = hostNF877
used/total:
left: CPU = 108.0/109.0, MEM = 75.0/78.0
right: CPU = 43.0/109.0, MEM = 76.0/78.0


In [120]:
a = {"a":1}
if "a" in a.keys():
    print(1)

1
